In [ ]:
import torch
import triton
import triton.language as tl

In [ ]:
@triton.jit
def layernorm_kernel_fwd(x_ptr, output_ptr, w_ptr, b_ptr, stride, n_cols, eps, BLOCK_SIZE: tl.constexpr):
    pid = tl.program_id(axis=0)
    offsets = tl.arange(0, BLOCK_SIZE)
    x_ptrs = x_ptr + pid * stride + offsets
    mask = offsets < n_cols
    x = tl.load(x_ptrs, mask=mask, other=0.)
    mean = tl.sum(x) / n_cols
    x_minus_mean = tl.where(offsets < n_cols, x - mean, 0.)
    var = tl.sum(x_minus_mean * x_minus_mean) / n_cols
    rstd = 1 / tl.sqrt(var + eps)
    x_hat = (x - mean) * rstd
    
    w = tl.load(w_ptr + offsets, mask=mask, other=0.)
    b = tl.load(b_ptr + offsets, mask=mask, other=0.)
    output = x_hat * w + b
    tl.store(output_ptr + pid * stride + offsets, output, mask=mask)

In [ ]:
def layer_norm(x, normalized_shape, weight, bias, eps):
    M, N = x.shape
    y = torch.empty_like(x)
    
    BLOCK_SIZE = triton.next_power_of_2(N)
    num_warps = min(max(BLOCK_SIZE // 256, 1), 8)

    layernorm_kernel_fwd[(M,)](x, y, weight, bias, x.stride(0), N, eps, BLOCK_SIZE=BLOCK_SIZE, num_warps=num_warps)
    return y

In [ ]:
def test_layer_norm(M, N, dtype, eps=1e-5, device='cuda'):
    x_shape = (M, N)
    w_shape = (x_shape[-1], )
    weight = torch.rand(w_shape, dtype=dtype, device=device)
    bias = torch.rand(w_shape, dtype=dtype, device=device)
    x = -2.3 + 0.5 * torch.randn(x_shape, dtype=dtype, device=device)
    y_tri = layer_norm(x, w_shape, weight, bias, eps)
    y_ref = torch.nn.functional.layer_norm(x, w_shape, weight, bias, eps).to(dtype)
    # print(y_tri)
    # print(y_ref)
    assert torch.allclose(y_tri, y_ref, atol=1e-2, rtol=0)

In [ ]:
@triton.testing.perf_report(
    triton.testing.Benchmark(
        x_names=['N'],  # argument names to use as an x-axis for the plot
        x_vals=[512 * i for i in range(2, 64)],  # different possible values for `x_name`
        line_arg='provider',  # argument name whose value corresponds to a different line in the plot
        line_vals=['triton', 'torch'],  # possible values for `line_arg``
        line_names=[
            "Triton",
            "Torch",
        ],  # label name for the lines
        styles=[('blue', '-'), ('green', '-')],  # line styles
        ylabel="GB/s",  # label name for the y-axis
        plot_name="layernorm-performance",  # name for the plot. Used also as a file name for saving the plot.
        args={'M': 4096, 'dtype': torch.float32, 'mode': 'forward'},  # values for function arguments not in `x_names` and `y_name`
    ))
def benchmark(M, N, dtype, provider, mode='forward', eps=1e-5, device='cuda'):
    # create data
    x_shape = (M, N)
    w_shape = (x_shape[-1], )
    weight = torch.rand(w_shape, dtype=dtype, device=device)
    bias = torch.rand(w_shape, dtype=dtype, device=device)
    x = -2.3 + 0.5 * torch.randn(x_shape, dtype=dtype, device=device)
    quantiles = [0.5, 0.2, 0.8]

    def y_fwd():
        if provider == 'torch':
            return layer_norm(x, w_shape, weight, bias, eps)
        if provider == 'triton':
            return torch.nn.functional.layer_norm(x, w_shape, weight, bias, eps)  
    
    if mode == 'forward':
        gbps = lambda ms: 2 * x.numel() * x.element_size() * 1e-9 / (ms * 1e-3)
        ms, min_ms, max_ms = triton.testing.do_bench(y_fwd, quantiles=quantiles, rep=500)
    
    return gbps(ms), gbps(max_ms), gbps(min_ms)

test_layer_norm(1151, 20000, torch.float32)
benchmark.run(show_plots=True, print_data=True)